In [1]:
%pip install gurobipy>=10
import gurobipy as grb
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats

# Q1: B&Q problem

In [2]:
model = grb.Model('B&Q')

I = 5

x={}
for i in range(I):
  x[i]= model.addVar(vtype=grb.GRB.INTEGER, name='x'+str(i))

x

Restricted license - for non-production use only - expires 2024-10-28


{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

##Constraints:
* 所有的工時加起來 <= 80 labor hours
* 長椅腳跟短椅腳加起來小於500*12(inches)
* 兩種椅子加起來的數量小於top的數量
* modelA的數量*4 = 短椅腳的數量
* modelB的數量*4 = 長椅腳的數量

In [3]:
model.addConstr((0.3*x[0] +0.3*x[1]+0.1*x[2]+0.15*x[3]+0.5*x[4]) <= 80)
model.addConstr((18*x[2]+ 30*x[3]) <= 500*12)
model.addConstr(x[0]+x[1]<=x[4])
model.addConstr(x[0]*4==x[2])
model.addConstr(x[1]*4==x[3])

<gurobi.Constr *Awaiting Model Update*>

## objective: 最大化收益30 * modelA + 45 * modelB

In [4]:
objective = 30*x[0] + 45 * x[1]
model.setObjective(objective, grb.GRB.MAXIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 5 columns and 14 nonzeros
Model fingerprint: 0x5163a7ee
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 6e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 1995.0000000

Root relaxation: objective 2.333333e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2333.3333

In [5]:
print(model.Status == grb.GRB.OPTIMAL)
print(model.display())

True
Maximize
  30.0 x0 + 45.0 x1
Subject To
  R0: 0.3 x0 + 0.3 x1 + 0.1 x2 + 0.15 x3 + 0.5 x4 <= 80
  R1: 18.0 x2 + 30.0 x3 <= 6000
  R2: x0 + x1 + -1.0 x4 <= 0
  R3: 4.0 x0 + -1.0 x2 = 0
  R4: 4.0 x1 + -1.0 x3 = 0
General Integers
  ['x0', 'x1', 'x2', 'x3', 'x4']
None


In [6]:
for v in model.getVars():
  print(v.VarName, v.X)

x0 28.0
x1 33.0
x2 112.0
x3 132.0
x4 61.0


# Q2: basketball team

In [7]:
m = grb.Model('Basketball')

I = 7

x={}
for i in range(I):
  x[i]= m.addVar(vtype=grb.GRB.INTEGER, name='x'+str(i))

x

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>}

0 = false
1 = true

In [8]:
G = [1, 0, 1, 0, 1, 0, 1]
C = [0, 1, 0, 1, 0, 1, 0]
F = [0, 0, 1, 1, 1, 1, 1]
BH = [3, 2, 2, 1, 3, 3, 3]
Shooting = [3, 1, 3, 3, 3, 1, 2]
Rebounding = [1, 3, 2, 3, 3, 2, 2]
Defense = [3, 2, 2, 1, 3, 3, 1]

## Constraints:
  * 至少4個人當G
  * 至少2個人當F
  * 至少1個人當C
  * ball-handling, shooting, rebounding平均評分至少高於2
  * 3號上的話6號就不上
  * 1號上的話4,5號一定要上
  * 2號或3號一定要上

In [9]:
m.addConstr(grb.quicksum(x[i] for i in range(I)) == 5)
for i in range(I):
  m.addConstr(x[i] <= 1)
  m.addConstr(x[i] >= 0)
m.addConstr(grb.quicksum(x[i] * G[i] for i in range(I))>=4)
m.addConstr(grb.quicksum(x[i] * F[i] for i in range(I))>=2)
m.addConstr(grb.quicksum(x[i] * C[i] for i in range(I))>=1)
#bh_mean = grb.quicksum(x[i] * BH[i] for i in range(I))/5
#sh_mean = grb.quicksum(x[i] * Shooting[i] for i in range(I))/5
#rb_mean = grb.quicksum(x[i] * Rebounding[i] for i in range(I))/5
m.addConstr((grb.quicksum((x[i] * BH[i]) + (x[i] * Shooting[i]) + (x[i] * Rebounding[i]) for i in range(I))/15) >= 2)
m.addConstr(x[2]+x[5] <= 1)
m.addConstr(x[3]+x[4]>=2*x[0])
m.addConstr(x[1]+x[2] >= 1)


<gurobi.Constr *Awaiting Model Update*>

## objective: 
* (max) total defense ability

In [10]:
obj = grb.quicksum(x[i] * Defense[i] for i in range(I))
m.setObjective(obj, grb.GRB.MAXIMIZE)
m.update()
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 7 columns and 47 nonzeros
Model fingerprint: 0xaabd71c8
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 22 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [11]:
print(m.Status == grb.GRB.OPTIMAL)
print(m.display())

True
Maximize
  3.0 x0 + 2.0 x1 + 2.0 x2 + x3 + 3.0 x4 + 3.0 x5 + x6
Subject To
  R0: x0 + x1 + x2 + x3 + x4 + x5 + x6 = 5
  R1: x0 <= 1
  R2: x0 >= 0
  R3: x1 <= 1
  R4: x1 >= 0
  R5: x2 <= 1
  R6: x2 >= 0
  R7: x3 <= 1
  R8: x3 >= 0
  R9: x4 <= 1
  R10: x4 >= 0
  R11: x5 <= 1
  R12: x5 >= 0
  R13: x6 <= 1
  R14: x6 >= 0
  R15: x0 + x2 + x4 + x6 >= 4
  R16: x2 + x3 + x4 + x5 + x6 >= 2
  R17: x1 + x3 + x5 >= 1
R18: 0.4666666666666667 x0 + 0.4 x1 + 0.4666666666666667 x2 + 0.4666666666666667 x3 +
 0.6000000000000001 x4 + 0.4 x5 + 0.4666666666666667 x6 >= 2
  R19: x2 + x5 <= 1
  R20: -2.0 x0 + x3 + x4 >= 0
  R21: x1 + x2 >= 1
General Integers
  ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']
None


In [12]:
for v in m.getVars():
  print(v.VarName, v.X)

x0 1.0
x1 0.0
x2 1.0
x3 1.0
x4 1.0
x5 0.0
x6 1.0
